In [1]:
import numpy as np
import pandas as pd 

def compute_equilibrium_ers(weights, sigma, risk_aversion):
    return risk_aversion * sigma @ weights

def compute_posterior_mean(mu, C, A, b, omega):
    c_inv = np.linalg.inv(C)
    a_t_oinv = A.T @ np.linalg.inv(omega)
 
    er = np.linalg.inv(c_inv + a_t_oinv @ A) @ (c_inv @ mu + a_t_oinv @ b)
 
    return pd.Series(er, index=mu.index)
 
def compute_posterior_cov(sigma, C, A, omega):
    return sigma + np.linalg.inv((np.linalg.inv(C) + A.T @ omega @ A))

In [2]:
# Example: US asset allocation

import yfinance as yf
 
tickers = ['VOX', 'VCR', 'VDC', 'VDE', 'VFH', 'VHT', 'VIS', 'VGT',
           'VAW', 'VNQ', 'VPU']
prices = yf.download(tickers, start='2018-01-01', end='2022-12-31')
rets = prices['Adj Close'].pct_change()
rets = rets.dropna(axis=0, how='any')[tickers]
sigma = rets.cov() * 252
 
weights = [.077, .107, .067, .048, .117, .143, .085, .273, .028, .027, .028]
weights = pd.Series(weights, index=tickers)
mu = compute_equilibrium_ers(weights, sigma, 1.4)

# Split portfolio to defensive sector and cyclical sector.

defensive = ['VDC', 'VHT', 'VPU']
cyclical = ['VCR', 'VDE', 'VFH', 'VIS', 'VGT', 'VAW']
 
defensive_port = weights[defensive] / weights[defensive].sum()
cyclical_port = weights[cyclical] / weights[cyclical].sum()
 
spread_port = pd.concat((defensive_port, -cyclical_port))
real_estate_port = pd.Series([1], index = ['VNQ'])
 
A = pd.DataFrame([spread_port, real_estate_port])
A = A.T.reindex(tickers).fillna(0).T
 
b = pd.Series([.01, .02])


tau = 0.05
omega = np.diag(np.diag(A @ (tau * sigma) @ A.T))
post_mean = compute_posterior_mean(mu, tau * sigma, A, b, omega)
post_cov = compute_posterior_cov(sigma, tau * sigma, A, omega)

[*********************100%%**********************]  11 of 11 completed


In [4]:
import cvxpy as cp
 
w = cp.Variable(len(mu))
market_vol = np.sqrt(weights @ sigma @ weights)
 
objective = cp.Maximize(post_mean.values @ w)
constraints = [cp.quad_form(w, sigma) <= market_vol ** 2,
               cp.sum(w) == 1,
               w >= 0]
prob = cp.Problem(objective, constraints)
result = prob.solve()
w = pd.Series(w.value, index=mu.index)
w

Ticker
VOX    3.262018e-08
VCR    2.872106e-08
VDC    2.420694e-08
VDE    6.357255e-08
VFH    3.911214e-03
VHT    5.805927e-01
VIS    5.716854e-08
VGT    4.154958e-01
VAW    8.043044e-08
VNQ    2.168046e-08
VPU    4.057979e-08
dtype: float64

In [ ]:
# Example: Global asset allocation including cryptocurrencies

tickers = ['VTI', 'VEA', 'VWO', 'AGG', 'BTC-USD', 'ETH-USD']
# Get weekly prices to avoid mismatch due to crypto being traded 24/7
prices = yf.download(tickers, start='2018-01-01', end='2022-12-31',         
                     interval='1wk')
prices = prices['Adj Close'].dropna(axis=0, how='any')[tickers]
rets = prices.pct_change()
sigma = rets.cov() * 52

weights = [.3401, .1559, .0708, .4269, .0045, .0018]
weights = pd.Series(weights, index=tickers)
mu = compute_equilibrium_ers(weights, sigma, 2.8)
 
btc_port = pd.Series([1], index = ['BTC-USD'])
eth_port = pd.Series([1], index = ['ETH-USD'])
A = pd.DataFrame([btc_port, eth_port])
A = A.T.reindex(tickers).fillna(0).T
b = pd.Series([.10, .15])
omega = np.diag([.05 ** 2, .05 ** 2])
 
tau = 0.05
post_mean = compute_posterior_mean(mu, tau * sigma, A, b, omega)
post_cov = compute_posterior_cov(sigma, tau * sigma, A, omega)